In [1]:
import sys
sys.path.append('../')
from Code.envs.statemachine import SuccessiveLookups
from Code.train import train, OptWrapper
from Code.everything5 import build_standard_model
import torch
from PIL import Image
import numpy as np

In [2]:
MAIN_DECAY = np.exp(-1/(20)*0.5)
ADAP_DECAY = np.exp(-1/(20*2))

spec = {
    'control_config': {
        'neuron_type': 'LIF',
        'n_neurons': 100,
        'BETA': 0.8,
        '1-beta': 'improved',
        'SPIKE_FN': 'bellec'
    },
    'mem_config': {
        'neuron_type': 'FlipFlop',
        'n_neurons': 20,
        'BETA': 0.8,
        '1-beta': 'improved',
        'SPIKE_FN': 'bellec',
        'ADAPSCALE': 30,
        'ADAPDECAY': None, #TODO: set this
        'OFFSET': None,
        'DECAY': MAIN_DECAY
    },
    'exp_config': {
        'n_sequence': 30,
        'val_sequence': 100,
        'round_length': 20
    },
    'lr': 0.001,
    'lr_decay': 1,
    'iterations': 5000,
    'batch_size': 64,
    'architecture': '1L'
}

In [3]:
DEVICE = torch.device('cuda')


train_problem = SuccessiveLookups(spec['iterations'], spec['batch_size'], spec['exp_config']['n_sequence'],
                               spec['exp_config']['round_length'], DEVICE)
val_problem = SuccessiveLookups(1, spec['batch_size'], spec['exp_config']['val_sequence'],
                               spec['exp_config']['round_length'], DEVICE)


n_in, n_out, input_rate = train_problem.get_infos()


In [4]:
model = build_standard_model(spec, n_in, input_rate, n_out)
model.to(DEVICE)

{'control': [[('input', 1), ('control', 1), ('mem', 1)], LIFNeuron(), <class 'torch.nn.modules.linear.Linear'>], 'mem': [[('input', 1), ('control', 1), ('mem', 1)], NewFlipFlop(
  (lif_on): LIFNeuron()
  (lif_off): LIFNeuron()
), <class 'torch.nn.modules.linear.Linear'>], 'output': [[('control', 1), ('mem', 1)], BaseNeuron(), None]}


OuterWrapper(
  (model): DynNetwork(
    (layers): ModuleDict(
      (loop): SequenceWrapper(
        (model): ParallelNetwork2(
          (layers): ModuleDict(
            (control): LIFNeuron()
            (control_synapse): Linear(in_features=148, out_features=100, bias=True)
            (mem): NewFlipFlop(
              (lif_on): LIFNeuron()
              (lif_off): LIFNeuron()
            )
            (mem_synapse): Linear(in_features=148, out_features=20, bias=True)
            (output): BaseNeuron()
          )
        )
      )
      (output_synapse): Linear(in_features=120, out_features=8, bias=True)
      (output): BaseNeuron()
    )
  )
)

In [5]:
optimizer = OptWrapper(model.parameters(), spec['lr'], spec['lr_decay'], 2500)

In [ ]:
train(train_problem, val_problem, optimizer, model, None)

Val Acc: 0.1284722238779068 Time per it: 4.824
It:   19 Loss: 2.082 Acc: 12.46%
It:   39 Loss: 2.078 Acc: 13.56%
It:   59 Loss: 2.075 Acc: 14.79%
It:   79 Loss: 2.052 Acc: 17.23%
It:   99 Loss: 2.037 Acc: 17.29%
Val Acc: 0.24021464586257935 Time per it: 2.020
It:  119 Loss: 2.013 Acc: 19.57%
It:  139 Loss: 1.991 Acc: 20.25%
It:  159 Loss: 1.929 Acc: 23.79%
It:  179 Loss: 1.985 Acc: 19.15%
It:  199 Loss: 1.953 Acc: 20.91%
Val Acc: 0.1974431872367859 Time per it: 2.013
It:  219 Loss: 2.010 Acc: 19.84%
It:  239 Loss: 1.930 Acc: 24.84%
It:  259 Loss: 1.866 Acc: 27.68%
It:  279 Loss: 1.872 Acc: 27.68%
It:  299 Loss: 1.852 Acc: 28.14%
Val Acc: 0.26799243688583374 Time per it: 2.065
It:  319 Loss: 1.838 Acc: 27.83%
It:  339 Loss: 1.798 Acc: 29.75%
It:  359 Loss: 1.755 Acc: 32.61%
It:  379 Loss: 1.746 Acc: 32.13%
It:  399 Loss: 1.830 Acc: 29.34%
Val Acc: 0.27320075035095215 Time per it: 2.083
It:  419 Loss: 1.867 Acc: 28.54%
It:  439 Loss: 1.837 Acc: 29.01%
It:  459 Loss: 1.776 Acc: 31.90%
It: